# Importar bibliotecas generales

In [ ]:
import numpy as np
import pandas as pd
import re as regex
import math as math

# Importar bibliotecas propias 

In [ ]:
#biblioteca para completar valores Nan de la columna rooms
%run "fill_column_rooms (entrega 1).ipynb

# Leer el data set desde el archivo y generar el data frame

In [ ]:
pathArchivoDataSet = 'properatti.csv'
df = pd.read_csv(pathArchivoDataSet)

# Detalle generales del data frame

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.columns

# La primer columna no tiene un nombre asignado

In [ ]:
type(df.iloc[0,0])

In [ ]:
len(df.loc[df.index == df['Unnamed: 0']])

## Para todas las filas del data frame, el valor de la primer columna es igual al valor del index. Se puede asumir que dicha columna corresponde a un campo ID. Se renombra la primer columna

In [ ]:
df.rename(columns={'Unnamed: 0': 'Id'}, inplace = True)
df.columns

# Limpieza del data set 

### Se eliminan columnas innecesarias

In [ ]:
#quitamos la columna floor, ya que es difícil de inferir a partir de las demás columnas, por simplicidad se quita de los datos
df.drop(labels='floor', axis = 1, inplace = True)

In [ ]:
#quitamos la columna image_thumbnail ya que no aporta información relevante
df = df.drop(labels='image_thumbnail', axis= 1)

In [ ]:
#quitamos la columna properati_url ya que no aporta información relevante
df = df.drop(labels='properati_url', axis= 1)

In [ ]:
#chequeamos las columnas que nos quedan
df.columns

### Se eliminan datos (filas) que no pudieron ser completados

In [ ]:
#se eliminan filas cuyo campo description está vacío, dicha columna se tomó como fuente para otras columnas
#al venir vacía otros datos no pueden ser completados
emptyDescriptionIndexes = df[df['description'].isnull()].index

#borramos las filas con los correspondientes indices
df.drop(emptyDescriptionIndexes , inplace=True)

#reseteamos los índices para poder seguir usándolos sin problemas
df.reset_index();

# Completar valores faltantes

### Completamos datos de coordenadas faltantes

##### Leemos directamente el archivo CSV generado en la notebook auxiliar "Completar coordenadas desde geonames_id" para completar valores de coordenadas faltantes en nuestro data frame. Para ver el proceso remitirse a dicha notebook

In [ ]:
latLngCSVFileName = 'latLngFromGeonames (entrega 1).csv'
#cargamos el archivo
latLongDF = pd.read_csv(latLngCSVFileName)

In [ ]:
df['lat'].isna().sum() #contamos los vacíos en lat

##### Definimos una función que toma una fila del data frame y le setea los datos de coordenadas si es que no los tiene

In [ ]:
def updateCoordinates(dataFrameRow):
    
    if(not math.isnan(dataFrameRow['geonames_id'])):
        geonameIdValue = int(dataFrameRow['geonames_id'])
        
        #usa el dataframe leido desde el archivo latLngFromGeonames.csv
        coordinatesData = latLongDF.loc[latLongDF['geonames_id'] == geonameIdValue]
        
        if(coordinatesData is not None and len(coordinatesData) == 1):
            dataFrameRow.lat = coordinatesData.lat
            dataFrameRow.lon = coordinatesData.lon
            dataFrameRow['lat-lon'] = coordinatesData['lat-lon']
    
    return dataFrameRow        

In [ ]:
#recorremos el data set aplicando el metodo para actualizar las coordenadas (demora un tiempo)
df = df.apply(updateCoordinates, axis=1)

In [ ]:
df['lat'].isna().sum() #contamos los vacíos en lat luego del cambio

### Datos de localidades: place_with_parent_names, place_name, country_name, state_name

In [ ]:
df[['place_with_parent_names', 'place_name','country_name','state_name']].isnull().sum()

##### Solo faltan 23 valores en la columna 'place_name' (información de barrio, zona, ciudad, etc.) que se intentarán obtener del la columna 'place_with_parent_names'.

In [ ]:
#seteamos la longitud del output para mejor lectura
pd.set_option('display.max_colwidth', -1)

##### Tomamos las columnas necesarias para completar los faltantes

In [ ]:
df[['place_with_parent_names', 'place_name','country_name','state_name']]

##### Se crea una nueva columna con los valores de la columna 'place_with_parent_names' en forma de lista

In [ ]:
def disgrega(valor):
    return valor.strip('|').split('|')

df['lista_auxiliar'] = df['place_with_parent_names'].apply(lambda x: disgrega(x))

In [ ]:
df['lista_auxiliar']

##### Se calcula cuantos elementos tiene cada lista

In [ ]:
df['conteo'] = df['lista_auxiliar'].apply(lambda x: len(x))
df['conteo']

##### Se determina cual es el mayor y menor números de elemntos en un registro de 'place_with_parent_names'

In [ ]:
df['conteo'].max()

In [ ]:
df['conteo'].min()

##### Se necesitarán 5 columnas como máximo para desempacar las distintas jerarquías de las locaciones. Mediante el comando apply se obtienen 5 columnas extras con información de localización por regiones.

In [ ]:
def chequeo2(valor):
    if len(valor) > 2:
        return valor[2]
    
    
def chequeo3(valor):
    if len(valor) > 3:
        return valor[3]
    
def chequeo4(valor):
    if len(valor) > 4:
        return valor[4]

df['loc1'] = df['lista_auxiliar'].apply(lambda x: x[0])
df['loc2'] = df['lista_auxiliar'].apply(lambda x: x[1])
df['loc3'] = df['lista_auxiliar'].apply(lambda x: chequeo2(x))
df['loc4'] = df['lista_auxiliar'].apply(lambda x: chequeo3(x))
df['loc5'] = df['lista_auxiliar'].apply(lambda x: chequeo4(x))

In [ ]:
daux = df[(df.loc5.notnull())]
daux[['loc1', 'loc2', 'loc3', 'loc4', 'loc5']]

##### Verificamos que los registros que tenían place_name vacíos tengan valores válidos en las nuevas columnas para cubrir el faltante

In [ ]:
len(df[df['place_name'].isna() & ~df['loc5'].isna()])

In [ ]:
df[df['place_name'].isna()][['lat','lon','geonames_id','place_name','place_with_parent_names', 'loc1', 'loc2', 'loc3', 'loc4', 'loc5']]

##### Completar el valor del campo rooms

In [ ]:
def completarValoresFaltantesEnFila(dataFrameRow):
    
    ##############################################################
    ##Actualizacion de Rooms
    updatedDataFameRow = updateRoomsFromRowData(dataFrameRow)
        
    return updatedDataFameRow

In [ ]:
df = df.apply(completarValoresFaltantesEnFila, axis=1);